In [59]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics as skm

from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

from sklearn.tree import DecisionTreeClassifier, plot_tree

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN

from keras.models import Sequential
from keras.layers import Dense, Dropout

from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasRegressor

rdst = 815

In [60]:
pets = pd.read_csv('../data/pets.csv')

In [61]:
pets.fillna(0, inplace = True)

In [62]:
pets.columns = map(str.lower, pets.columns)

In [63]:
pets.drop(pets[pets['quantity'] > 3].index, inplace = True) 

In [64]:
pets = pd.get_dummies(pets, columns = ['breed1','breed2','color1', 'color2', 'color3','type', 'gender'], drop_first = True)

In [65]:
pets.isnull().sum().sum()

0

In [66]:
# Need to convert the adoption speed to numericals that reflect the timing
pets['adoptionspeed'].replace({
0:1,
1:7,
2:20,
3:60,
4:100,
}, inplace = True)
pets['adoptionspeed'][:5]

0    100
1    100
2     60
3    100
4     20
Name: adoptionspeed, dtype: int64

In [67]:
# Train-Test Split  - 7686x323 and 2563x323

X = pets.drop(columns = ['name', 'adoptionspeed', 'description', 'petid', 'rescuerid', 'state'])
y = pets['adoptionspeed']

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = rdst)

print('X Train', X_train.shape)
print('X Test', X_test.shape)
print()
print('y Train', y_train.shape)
print('y Test', y_test.shape)


X Train (7686, 309)
X Test (2563, 309)

y Train (7686,)
y Test (2563,)


In [68]:
# Standard Scalar

ss = StandardScaler()
Z_train = ss.fit_transform(X_train)
Z_test  = ss.transform(X_test)
print('Z Train', Z_train.shape)
print('Z Test', Z_test.shape)


Z Train (7686, 309)
Z Test (2563, 309)


In [69]:
# Baseline:
y.mean()

47.40472241194263

This assumes 47 days based on how we assumed the days for each category.

In [70]:
# Linear Regression
lr = LinearRegression()
lr.fit(Z_train,y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [71]:
lr.score(Z_train, y_train)

0.11398223669250675

In [72]:
preds = lr.predict(Z_test)
resids = y_test - preds
preds = pd.DataFrame(preds)

In [73]:
preds.sort_values(by = 0, ascending = False)

0
2520  1.051038e+15
539   7.153490e+14
1814  7.104476e+14
1556  7.104476e+14
840   5.824343e+14
...            ...
1816 -3.139657e+14
732  -3.139657e+14
1207 -3.969319e+14
332  -6.007746e+14
2237 -8.507493e+14

[2563 rows x 1 columns]

In [74]:
type(resids)

pandas.core.series.Series

In [75]:
knn = KNeighborsRegressor(n_neighbors = 5)
cross_val_score(knn, Z_train, y_train, cv =5).mean()

-0.07582280714709588

In [76]:
# Fit KNN
knn.fit(Z_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

In [77]:
print("Train Score: ", knn.score(Z_train, y_train))
print('Test Score: ',  knn.score(Z_test, y_test))
print('Cross-Val: ',   cross_val_score(knn,Z_train, y_train).mean())

Train Score:  0.28771112701360313
Test Score:  -0.04591662753333381
Cross-Val:  -0.07582280714709588


In [78]:
dt = DecisionTreeRegressor(random_state = rdst)
dt.fit(X_train, y_train)
print(f'Score on training set: {dt.score(X_train, y_train)}')
print(f'Score on testing set: {dt.score(X_test, y_test)}')

Score on training set: 0.9798150781475911
Score on testing set: -0.6238822882903479


In [79]:
grid = GridSearchCV(DecisionTreeRegressor(),
                   param_grid = {'max_depth' : [2,3,4,5,6,7,8,9],
                                 'min_samples_split' : [5, 10, 15, 20, 25, 30],
                                 'min_samples_leaf' : [2, 3, 4, 5, 6, 7, 8, 9]},
                                cv = 5,
                                 verbose = 2) # to see it work
                                

In [80]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 384 candidates, totalling 1920 fits
[CV] max_depth=2, min_samples_leaf=2, min_samples_split=5 ............
[CV]  max_depth=2, min_samples_leaf=2, min_samples_split=5, total=   0.1s
[CV] max_depth=2, min_samples_leaf=2, min_samples_split=5 ............
[CV]  max_depth=2, min_samples_leaf=2, min_samples_split=5, total=   0.0s
[CV] max_depth=2, min_samples_leaf=2, min_samples_split=5 ............
[CV]  max_depth=2, min_samples_leaf=2, min_samples_split=5, total=   0.0s
[CV] max_depth=2, min_samples_leaf=2, min_samples_split=5 ............


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  max_depth=2, min_samples_leaf=2, min_samples_split=5, total=   0.0s
[CV] max_depth=2, min_samples_leaf=2, min_samples_split=5 ............
[CV]  max_depth=2, min_samples_leaf=2, min_samples_split=5, total=   0.0s
[CV] max_depth=2, min_samples_leaf=2, min_samples_split=10 ...........
[CV]  max_depth=2, min_samples_leaf=2, min_samples_split=10, total=   0.1s
[CV] max_depth=2, min_samples_leaf=2, min_samples_split=10 ...........
[CV]  max_depth=2, min_samples_leaf=2, min_samples_split=10, total=   0.0s
[CV] max_depth=2, min_samples_leaf=2, min_samples_split=10 ...........
[CV]  max_depth=2, min_samples_leaf=2, min_samples_split=10, total=   0.0s
[CV] max_depth=2, min_samples_leaf=2, min_samples_split=10 ...........
[CV]  max_depth=2, min_samples_leaf=2, min_samples_split=10, total=   0.0s
[CV] max_depth=2, min_samples_leaf=2, min_samples_split=10 ...........
[CV]  max_depth=2, min_samples_leaf=2, min_samples_split=10, total=   0.0s
[CV] max_depth=2, min_samples_leaf=2, min_samples_s

[CV]  max_depth=2, min_samples_leaf=4, min_samples_split=5, total=   0.0s
[CV] max_depth=2, min_samples_leaf=4, min_samples_split=5 ............
[CV]  max_depth=2, min_samples_leaf=4, min_samples_split=5, total=   0.0s
[CV] max_depth=2, min_samples_leaf=4, min_samples_split=10 ...........
[CV]  max_depth=2, min_samples_leaf=4, min_samples_split=10, total=   0.1s
[CV] max_depth=2, min_samples_leaf=4, min_samples_split=10 ...........
[CV]  max_depth=2, min_samples_leaf=4, min_samples_split=10, total=   0.0s
[CV] max_depth=2, min_samples_leaf=4, min_samples_split=10 ...........
[CV]  max_depth=2, min_samples_leaf=4, min_samples_split=10, total=   0.1s
[CV] max_depth=2, min_samples_leaf=4, min_samples_split=10 ...........
[CV]  max_depth=2, min_samples_leaf=4, min_samples_split=10, total=   0.0s
[CV] max_depth=2, min_samples_leaf=4, min_samples_split=10 ...........
[CV]  max_depth=2, min_samples_leaf=4, min_samples_split=10, total=   0.0s
[CV] max_depth=2, min_samples_leaf=4, min_samples_s

[CV]  max_depth=2, min_samples_leaf=6, min_samples_split=5, total=   0.1s
[CV] max_depth=2, min_samples_leaf=6, min_samples_split=5 ............
[CV]  max_depth=2, min_samples_leaf=6, min_samples_split=5, total=   0.0s
[CV] max_depth=2, min_samples_leaf=6, min_samples_split=5 ............
[CV]  max_depth=2, min_samples_leaf=6, min_samples_split=5, total=   0.0s
[CV] max_depth=2, min_samples_leaf=6, min_samples_split=10 ...........
[CV]  max_depth=2, min_samples_leaf=6, min_samples_split=10, total=   0.0s
[CV] max_depth=2, min_samples_leaf=6, min_samples_split=10 ...........
[CV]  max_depth=2, min_samples_leaf=6, min_samples_split=10, total=   0.0s
[CV] max_depth=2, min_samples_leaf=6, min_samples_split=10 ...........
[CV]  max_depth=2, min_samples_leaf=6, min_samples_split=10, total=   0.1s
[CV] max_depth=2, min_samples_leaf=6, min_samples_split=10 ...........
[CV]  max_depth=2, min_samples_leaf=6, min_samples_split=10, total=   0.0s
[CV] max_depth=2, min_samples_leaf=6, min_samples_sp

...

[CV]  max_depth=9, min_samples_leaf=4, min_samples_split=25, total=   0.1s
[CV] max_depth=9, min_samples_leaf=4, min_samples_split=30 ...........
[CV]  max_depth=9, min_samples_leaf=4, min_samples_split=30, total=   0.1s
[CV] max_depth=9, min_samples_leaf=4, min_samples_split=30 ...........
[CV]  max_depth=9, min_samples_leaf=4, min_samples_split=30, total=   0.1s
[CV] max_depth=9, min_samples_leaf=4, min_samples_split=30 ...........
[CV]  max_depth=9, min_samples_leaf=4, min_samples_split=30, total=   0.1s
[CV] max_depth=9, min_samples_leaf=4, min_samples_split=30 ...........
[CV]  max_depth=9, min_samples_leaf=4, min_samples_split=30, total=   0.1s
[CV] max_depth=9, min_samples_leaf=4, min_samples_split=30 ...........
[CV]  max_depth=9, min_samples_leaf=4, min_samples_split=30, total=   0.1s
[CV] max_depth=9, min_samples_leaf=5, min_samples_split=5 ............
[CV]  max_depth=9, min_samples_leaf=5, min_samples_split=5, total=   0.1s
[CV] max_depth=9, min_samples_leaf=5, min_samples_

[CV]  max_depth=9, min_samples_leaf=6, min_samples_split=25, total=   0.1s
[CV] max_depth=9, min_samples_leaf=6, min_samples_split=25 ...........
[CV]  max_depth=9, min_samples_leaf=6, min_samples_split=25, total=   0.1s
[CV] max_depth=9, min_samples_leaf=6, min_samples_split=25 ...........
[CV]  max_depth=9, min_samples_leaf=6, min_samples_split=25, total=   0.1s
[CV] max_depth=9, min_samples_leaf=6, min_samples_split=25 ...........
[CV]  max_depth=9, min_samples_leaf=6, min_samples_split=25, total=   0.1s
[CV] max_depth=9, min_samples_leaf=6, min_samples_split=30 ...........
[CV]  max_depth=9, min_samples_leaf=6, min_samples_split=30, total=   0.1s
[CV] max_depth=9, min_samples_leaf=6, min_samples_split=30 ...........
[CV]  max_depth=9, min_samples_leaf=6, min_samples_split=30, total=   0.1s
[CV] max_depth=9, min_samples_leaf=6, min_samples_split=30 ...........
[CV]  max_depth=9, min_samples_leaf=6, min_samples_split=30, total=   0.1s
[CV] max_depth=9, min_samples_leaf=6, min_samples

[CV]  max_depth=9, min_samples_leaf=8, min_samples_split=20, total=   0.1s
[CV] max_depth=9, min_samples_leaf=8, min_samples_split=20 ...........
[CV]  max_depth=9, min_samples_leaf=8, min_samples_split=20, total=   0.1s
[CV] max_depth=9, min_samples_leaf=8, min_samples_split=25 ...........
[CV]  max_depth=9, min_samples_leaf=8, min_samples_split=25, total=   0.1s
[CV] max_depth=9, min_samples_leaf=8, min_samples_split=25 ...........
[CV]  max_depth=9, min_samples_leaf=8, min_samples_split=25, total=   0.1s
[CV] max_depth=9, min_samples_leaf=8, min_samples_split=25 ...........
[CV]  max_depth=9, min_samples_leaf=8, min_samples_split=25, total=   0.1s
[CV] max_depth=9, min_samples_leaf=8, min_samples_split=25 ...........
[CV]  max_depth=9, min_samples_leaf=8, min_samples_split=25, total=   0.1s
[CV] max_depth=9, min_samples_leaf=8, min_samples_split=25 ...........
[CV]  max_depth=9, min_samples_leaf=8, min_samples_split=25, total=   0.1s
[CV] max_depth=9, min_samples_leaf=8, min_samples

[Parallel(n_jobs=1)]: Done 1920 out of 1920 | elapsed:  2.0min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse',
                                             max_depth=None, max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort='deprecated',
                                             random_state=None,
                                             splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': [2, 3, 4, 5, 6, 7, 8, 9],
                         'min_samples_leaf': [2, 3, 4, 5, 6, 7, 8, 9],
                    

In [81]:
dec_tree = grid.best_estimator_
dec_tree

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=5,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=3, min_samples_split=30,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [82]:
print('Train Score: ', grid.score(X_train, y_train))
print('Test Score: ', grid.score(X_test, y_test))
print('Cross Val: ', cross_val_score(dec_tree, X_train, y_train).mean())

Train Score:  0.1714087281962856
Test Score:  0.123552392773709
Cross Val:  0.13561603853276655


Yikes, those are some bad scores!

## Neural Network

In [83]:
Z_train.shape[1]

309

In [85]:
def model_fn(layer_one_neurons = 32, layer_two_neurons = 32, layer_one_dropout = .5):
    model = Sequential()
    model.add(Dense(layer_one_neurons, 
                    activation = 'relu', 
                    input_shape = (Z_train.shape[1],)))
    model.add(Dropout(layer_one_dropout))
    model.add(Dense(layer_two_neurons, activation = 'relu'))
    model.add(Dense(1, activation = None))
    model.compile(loss = 'mse', optimizer = 'adam')
    return model

nn = KerasRegressor(build_fn = model_fn, batch_size = 512, verbose = 1)
nn_params = {
    'epochs' : [100,500],
    'layer_one_neurons' : [500, 1_000],
    'layer_one_dropout' : [.5],
    'layer_two_neurons'  : [64, 128]}
gs = GridSearchCV(nn, param_grid = nn_params, cv = 3)
gs.fit(Z_train, y_train)

print(gs.best_score_)
gs.best_params_


Epoch 1/100
5124/5124 [==============================] - 0s 42us/step - loss: 3306.6663
Epoch 2/100
5124/5124 [==============================] - 0s 41us/step - loss: 2681.1881
Epoch 3/100
5124/5124 [==============================] - 0s 39us/step - loss: 2152.8801
Epoch 4/100
5124/5124 [==============================] - 0s 41us/step - loss: 1747.6888
Epoch 5/100
5124/5124 [==============================] - 0s 38us/step - loss: 1491.3322
Epoch 6/100
5124/5124 [==============================] - 0s 41us/step - loss: 1372.3086
Epoch 7/100
5124/5124 [==============================] - 0s 47us/step - loss: 1315.1044
Epoch 8/100
5124/5124 [==============================] - 0s 39us/step - loss: 1267.9751
Epoch 9/100
5124/5124 [==============================] - 0s 43us/step - loss: 1251.1873
Epoch 10/100
5124/5124 [==============================] - 0s 39us/step - loss: 1235.2174
Epoch 11/100
5124/5124 [==============================] - 0s 37us/step - loss: 1230.4494
Epoch 12/100
5124/5124 [======

5124/5124 [==============================] - 0s 47us/step - loss: 1059.6410
Epoch 94/100
5124/5124 [==============================] - 0s 39us/step - loss: 1053.6181
Epoch 95/100
5124/5124 [==============================] - 0s 36us/step - loss: 1044.9977
Epoch 96/100
5124/5124 [==============================] - 0s 42us/step - loss: 1035.6747
Epoch 97/100
5124/5124 [==============================] - 0s 40us/step - loss: 1044.2410
Epoch 98/100
5124/5124 [==============================] - 0s 37us/step - loss: 1054.6853
Epoch 99/100
5124/5124 [==============================] - 0s 39us/step - loss: 1046.1195
Epoch 100/100
2562/2562 [==============================] - 0s 24us/step
Epoch 1/100
5124/5124 [==============================] - 0s 38us/step - loss: 3466.1149
Epoch 2/100
5124/5124 [==============================] - 0s 39us/step - loss: 2939.9652
Epoch 3/100
5124/5124 [==============================] - 0s 39us/step - loss: 2394.6084
Epoch 4/100
5124/5124 [==============================]

5124/5124 [==============================] - 0s 47us/step - loss: 1062.8646
Epoch 85/100
5124/5124 [==============================] - 0s 48us/step - loss: 1058.0989
Epoch 86/100
5124/5124 [==============================] - 0s 55us/step - loss: 1051.6143
Epoch 87/100
5124/5124 [==============================] - 0s 50us/step - loss: 1059.4313
Epoch 88/100
5124/5124 [==============================] - 0s 45us/step - loss: 1069.2108
Epoch 89/100
5124/5124 [==============================] - 0s 50us/step - loss: 1057.4008
Epoch 90/100
5124/5124 [==============================] - 0s 42us/step - loss: 1049.6744
Epoch 91/100
5124/5124 [==============================] - 0s 42us/step - loss: 1048.0418
Epoch 92/100
5124/5124 [==============================] - 0s 48us/step - loss: 1041.9727
Epoch 93/100
5124/5124 [==============================] - 0s 48us/step - loss: 1048.3130
Epoch 94/100
5124/5124 [==============================] - 0s 48us/step - loss: 1042.9247
Epoch 95/100
5124/5124 [==========

5124/5124 [==============================] - 0s 46us/step - loss: 1084.0286
Epoch 76/100
5124/5124 [==============================] - 0s 45us/step - loss: 1091.2744
Epoch 77/100
5124/5124 [==============================] - 0s 47us/step - loss: 1086.2771
Epoch 78/100
5124/5124 [==============================] - 0s 41us/step - loss: 1087.7517
Epoch 79/100
5124/5124 [==============================] - 0s 43us/step - loss: 1081.7023
Epoch 80/100
5124/5124 [==============================] - 0s 41us/step - loss: 1074.9170
Epoch 81/100
5124/5124 [==============================] - 0s 43us/step - loss: 1085.0735
Epoch 82/100
5124/5124 [==============================] - 0s 47us/step - loss: 1077.2478
Epoch 83/100
5124/5124 [==============================] - 0s 41us/step - loss: 1078.4391
Epoch 84/100
5124/5124 [==============================] - 0s 45us/step - loss: 1068.8110
Epoch 85/100
5124/5124 [==============================] - 0s 41us/step - loss: 1070.0738
Epoch 86/100
5124/5124 [==========

5124/5124 [==============================] - 0s 47us/step - loss: 1096.5128
Epoch 67/100
5124/5124 [==============================] - 0s 46us/step - loss: 1091.0601
Epoch 68/100
5124/5124 [==============================] - 0s 47us/step - loss: 1094.0116
Epoch 69/100
5124/5124 [==============================] - 0s 51us/step - loss: 1101.4355
Epoch 70/100
5124/5124 [==============================] - 0s 45us/step - loss: 1100.4988
Epoch 71/100
5124/5124 [==============================] - 0s 49us/step - loss: 1093.2279
Epoch 72/100
5124/5124 [==============================] - 0s 47us/step - loss: 1081.4994
Epoch 73/100
5124/5124 [==============================] - 0s 46us/step - loss: 1081.3423
Epoch 74/100
5124/5124 [==============================] - 0s 49us/step - loss: 1083.3901
Epoch 75/100
5124/5124 [==============================] - 0s 50us/step - loss: 1081.3368
Epoch 76/100
5124/5124 [==============================] - 0s 52us/step - loss: 1099.6050
Epoch 77/100
5124/5124 [==========

...

5124/5124 [==============================] - 2s 391us/step - loss: 824.6465
Epoch 287/500
5124/5124 [==============================] - 3s 504us/step - loss: 822.5707
Epoch 288/500
5124/5124 [==============================] - 3s 519us/step - loss: 830.9587
Epoch 289/500
5124/5124 [==============================] - 2s 462us/step - loss: 830.1729
Epoch 290/500
5124/5124 [==============================] - 2s 428us/step - loss: 812.0915
Epoch 291/500
5124/5124 [==============================] - 3s 615us/step - loss: 820.9087
Epoch 292/500
5124/5124 [==============================] - 2s 413us/step - loss: 814.0635
Epoch 293/500
5124/5124 [==============================] - 2s 481us/step - loss: 818.4414
Epoch 294/500
5124/5124 [==============================] - 3s 517us/step - loss: 819.2265
Epoch 295/500
5124/5124 [==============================] - 3s 613us/step - loss: 821.4403
Epoch 296/500
5124/5124 [==============================] - 2s 445us/step - loss: 807.8578
Epoch 297/500
5124/5124 

5124/5124 [==============================] - 2s 465us/step - loss: 759.1316
Epoch 378/500
5124/5124 [==============================] - 2s 420us/step - loss: 776.9885
Epoch 379/500
5124/5124 [==============================] - 2s 437us/step - loss: 753.6796
Epoch 380/500
5124/5124 [==============================] - 3s 518us/step - loss: 759.1759
Epoch 381/500
5124/5124 [==============================] - 2s 443us/step - loss: 748.4121
Epoch 382/500
5124/5124 [==============================] - 2s 470us/step - loss: 747.9085
Epoch 383/500
5124/5124 [==============================] - 2s 451us/step - loss: 765.7126
Epoch 384/500
5124/5124 [==============================] - 3s 547us/step - loss: 759.1252
Epoch 385/500
5124/5124 [==============================] - 2s 433us/step - loss: 777.3967 0s - loss: 783.50
Epoch 386/500
5124/5124 [==============================] - 2s 487us/step - loss: 763.2022
Epoch 387/500
5124/5124 [==============================] - 3s 493us/step - loss: 761.9511
Epoch 

5124/5124 [==============================] - 3s 555us/step - loss: 715.7488
Epoch 468/500
5124/5124 [==============================] - 2s 407us/step - loss: 722.9487
Epoch 469/500
5124/5124 [==============================] - 2s 423us/step - loss: 709.1178
Epoch 470/500
5124/5124 [==============================] - 2s 417us/step - loss: 708.4396
Epoch 471/500
5124/5124 [==============================] - 2s 429us/step - loss: 707.2677
Epoch 472/500
5124/5124 [==============================] - 2s 437us/step - loss: 701.4895
Epoch 473/500
5124/5124 [==============================] - 3s 511us/step - loss: 715.1615
Epoch 474/500
5124/5124 [==============================] - 3s 497us/step - loss: 707.5463
Epoch 475/500
5124/5124 [==============================] - 2s 439us/step - loss: 701.7392 0s - loss: 703.
Epoch 476/500
5124/5124 [==============================] - 2s 395us/step - loss: 715.5177
Epoch 477/500
5124/5124 [==============================] - 2s 419us/step - loss: 721.1251
Epoch 47

7686/7686 [==============================] - 4s 482us/step - loss: 1013.9517
Epoch 58/100
7686/7686 [==============================] - 4s 575us/step - loss: 1008.9288
Epoch 59/100
7686/7686 [==============================] - 4s 504us/step - loss: 1016.9081
Epoch 60/100
7686/7686 [==============================] - 3s 414us/step - loss: 1018.4381
Epoch 61/100
7686/7686 [==============================] - 4s 456us/step - loss: 1017.7670
Epoch 62/100
7686/7686 [==============================] - 4s 500us/step - loss: 999.2720
Epoch 63/100
7686/7686 [==============================] - 3s 442us/step - loss: 1001.6774
Epoch 64/100
7686/7686 [==============================] - 3s 417us/step - loss: 1004.2240
Epoch 65/100
7686/7686 [==============================] - 3s 401us/step - loss: 993.4790
Epoch 66/100
7686/7686 [==============================] - 4s 492us/step - loss: 992.2040
Epoch 67/100
7686/7686 [==============================] - 4s 476us/step - loss: 994.0413
Epoch 68/100
7686/7686 [===

{'epochs': 100,
 'layer_one_dropout': 0.5,
 'layer_one_neurons': 1000,
 'layer_two_neurons': 128}

In [86]:
kr_best = gs.best_score_
gs.best_params_

{'epochs': 100,
 'layer_one_dropout': 0.5,
 'layer_one_neurons': 1000,
 'layer_two_neurons': 128}

In [87]:
np.sqrt(-kr_best)

36.519939036596426